## 4.5 GBM(Gradient Boosting Machine)

**부스팅 알고리즘**  
1. 개념: 여러 개의 약한 학습기(weak learner)를 순차적으로 학습-예측하면서 잘못 예측한 데이터에 가중치 부여를 통해 오류를 개선해 나가면서 학습하는 방식  


2. 대표적인 예  
    - AdaBoost(Adaptive boosting) : 오류 데이터에 가중치를 부여하면서 부스팅을 수행하는 대표적인 알고리즘  
    - 그래디언트 부스트  

1) AdaBoost  
<img src='https://blog.kakaocdn.net/dn/bZaE8F/btqzOlQjTa3/Yadk0nZsSokfCd2OUXUM3k/img.jpg'>

2) GBM(Gradient Boost Machine)  
- 에이다부스트와 유사하나, <u>가중치 업데이트를 경사 하강법(Gradient Descent)을 이용</u>하는 것이 큰 차이이다.  
- 오류 값은 실제값 - 예측값  
<center> y : 실제 결과값, $x_1,x_2,...,x_n$ : 피처, $F(x)$ : 피처에 기반한 예측 함수 </center>

$$오류식: h(x) = y-F(x)$$  

=> 오류 식을 최소화하는 방향성을 가지고 반복적으로 가중치 값으로 업데이트 하는 것 : **경사 하강법**

In [4]:
import pandas as pd

def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) 
                                                                                           if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

In [6]:
import pandas as pd

def get_human_dataset( ):
    
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv('./human_activity/features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])
    
    # 중복된 feature명을 새롭게 수정하는 get_new_feature_name_df()를 이용하여 새로운 feature명 DataFrame생성. 
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    # 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
    X_train = pd.read_csv('./human_activity/train/X_train.txt',sep='\s+', names=feature_name )
    X_test = pd.read_csv('./human_activity/test/X_test.txt',sep='\s+', names=feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv('./human_activity/train/y_train.txt',sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv('./human_activity/test/y_test.txt',sep='\s+',header=None,names=['action'])
    
    # 로드된 학습/테스트용 DataFrame을 모두 반환 
    return X_train, X_test, y_train, y_test

In [8]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time
import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = get_human_dataset()

# GBM 수행 시간 측정을 위함. 시작 시간 설정
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train,y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print('GBM 정확도: {0:.4f}'.format(gb_accuracy))
print('GBM 수행 시간: {0:.1f}'.format(time.time() - start_time))

GBM 정확도: 0.9382
GBM 수행 시간: 398.8


기본 하이퍼 파라미터만으로 93.76%의 예측 정확도로 앞의 랜덤 포레스트보다 나은 예측 성능을 나타낸다. 일반적으로 GBM이 랜덤 포레스트보다는 예측 성능이 조금 뛰어난 경우가 많다. 


그러나 수행 시간이 오래 걸리고, 하이퍼 파라미터 튜닝 노력도 더 필요하다. 

### GBM 하이퍼 파라미터 및 튜닝

**하이퍼 파라미터**
1. loss  
    - 경사하강법에서 사용할 비용 함수를 지정  
    - 기본값 : 'deviance'
    
    
2. learning_rate  
    - GBM이 학습을 진행할 때마다 적용하는 학습률  
    - Weak learner가 순차적으로 오류 값을 보정해 나가는 데 적용하는 개수  
    - 0~1 사이의 값을 지정할 수 있으며 기본값은 1이다.  
    - learning_rate는 n_estimators와 상호 보완적으로 조합해 사용한다.  
  
  
3. n_estimators  
    - weak learner의 개수  
    - weak learner가 순차적으로 오류를 보정하므로 개수가 많을수록 예측 성능이 일정 수준까지는 좋아질 수 있다. 하지만 개수가 많을수록 수행 시간이 오래 걸린다.  
    - 기본값은 100이다.  
    

4. subsample  
    - weak learner가 학습에 사용하는 데이터의 샘플링 비율  
    - 기본값은 1이며, 이는 전체 학습 데이터를 기반으로 학습한다는 의미이다.  
    - 0.5이면 학습 데이터의 50%, 과적합이 염려되는 경우 subsample을 1보다 작은 값으로 설정한다.  

In [9]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators' :[100,500],
    'learning_rate' : [0.05,0.1]
}

grid_cv = GridSearchCV(gb_clf, param_grid=params, cv=2, verbose=1)
grid_cv.fit(X_train, y_train)
print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 39.6min finished


최적 하이퍼 파라미터:
 {'learning_rate': 0.05, 'n_estimators': 500}
최고 예측 정확도: 0.9013


learning_rate가 0.05, n_estimators가 500일 때 2개의 교차 검증 세트에서 90.1% 정확도가 최고로 도출  

In [ ]:
# GridSearchCV를 이용해 최적으로 학습된 estimator로 예측 수행
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print('GBM 정확도: {0:.4f}'.format(gb_accuracy))

테스트 데이터 세트에서 약 94.1%의 정확도이다. 


GBM은 과적합에도 강한 뛰어난 예측 성능을 가진 알고리즘이지만, 수행 시간이 오래 걸린다는 단점이 있다.  
+) GBM을 기반 ML 패키지는 XGBoost와 LightGBM이다.